In [ ]:
import random
import copy
import networkx as nx
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import math
import pickle
import os
import time
from datetime import datetime
from scipy.stats import qmc
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
random.seed(datetime.now().timestamp())

In [ ]:
!unzip /content/gdrive/MyDrive/Networks/test_cases.zip -d /

In [ ]:
num_of_nodes = [20, 25, 30, 35, 40, 45, 50, 55, 60]
test_case_num = 30

#defining network parameters
GATEWAY_ID = 0
spread_factor = range(7,13)

In [ ]:
flock_size = 30
max_iterations = 500
large_positive_value = 10**6
large_negative_value = -10**6
init_priority = [-1000,1000]
C = 0.5
alpha = 1
beta = 2
awareness = 0.1
flight_length = 2


In [ ]:
class Crows:
  def __init__(self, position, fitness):
    self.position = position
    self.fitness  = fitness
  def __str__(self):
    return f'''
            position = {self.position}\n
            fitness = {self.fitness}\n
            '''

In [ ]:
def dist(a,b):
  x1,y1 = a
  x2,y2 = b
  return math.sqrt((x2-x1)**2+(y2-y1)**2)

In [ ]:
def read_data(path, isGraph):
  if isGraph:
    G = nx.read_gml(path,destringizer=int)
    return G
  with open(path,'rb') as file:
    data = pickle.load(file)
    file.close()
    return data

In [ ]:
def decode_path(G, position, edge_nodes):
  N_DEVICES = len(G.nodes)
  ans = {}
  for f in edge_nodes:
    iteration = 0
    visited_nodes = [False for u in range(N_DEVICES)]
    start = {(f,k): position[f,k] for k in spread_factor}
    terminal_node, sf = max(start, key = lambda x: start[x])
    path = []
    while terminal_node != GATEWAY_ID and iteration <= N_DEVICES:
      visited_nodes[terminal_node] = True
      adj_nodes = {}
      for v in G.adj[terminal_node]:
        if visited_nodes[v]:
          continue
        for k in spread_factor:
          adj_nodes[(v,k)] = position[v,k]
      if len(adj_nodes) == 0:
        break
      next_node, next_sf = max(adj_nodes, key = lambda x: adj_nodes[x])
      path.append((terminal_node, next_node, sf))
      terminal_node = next_node
      sf = next_sf
      iteration = iteration + 1
    ans[f] = path
  return ans

In [ ]:
def calculate_fitness(crow, iteration, edge_nodes, G, network_data):
  fitness_cost = 0
  path = decode_path(G, crow.position, edge_nodes)
  for f in edge_nodes:
    #adding a penalty if invalid path in returned
    if path[f][-1][1] != GATEWAY_ID:
      fitness_cost = fitness_cost + large_positive_value
      continue
    
    #Calculate cost of path and adding link capacity penalty
    total_delay = 0
    for i,j,k in path[f]:
      fitness_cost = fitness_cost + network_data['cst'][f,i,j,k]
      if network_data['max_edge_data_rate'][k]-network_data['data_rate'][f,i,j,k] < 0:
        fitness_cost = fitness_cost + network_data['cst'][f,i,j,k] + ((C*iteration)**alpha)*((network_data['max_edge_data_rate'][k]-network_data['data_rate'][f,i,j,k])**beta) 
      total_delay = total_delay + network_data['edge_delays'][f,i,j] + network_data['tx_time'][k] + network_data['eh_time'][i,k]      

    #Adding delay constraint penalty
    if network_data['max_flow_delays'][f]-total_delay < 0:
      fitness_cost = fitness_cost + ((C*iteration)**alpha)*((network_data['max_flow_delays'][f]-total_delay)**beta)

  crow.fitness = fitness_cost

In [ ]:
def Crow_Search(edge_nodes, G, network_data):
  start = time.time()
  end = time.time()
  converge_iter = 0
  N_DEVICES = len(G.nodes)

  #Random initialization
  crow_flock = [Crows(
      {(u,k) : random.randint(*init_priority) for u in range(N_DEVICES) for k in spread_factor},
      0
  ) for c in range(flock_size)]


  #evaluate the position of crows
  for c in range(flock_size):
    calculate_fitness(crow_flock[c], 0, edge_nodes, G, network_data)
  
  #initialize best postion
  best_crow = copy.deepcopy(crow_flock[0])

  crow_memory = []
  #initialize memory of each crow
  for c in range(flock_size):
    crow_memory.append(copy.deepcopy(crow_flock[c]))

  for iter in range(1,max_iterations+1):
    for c in range(flock_size):

      #randomly choose one of the crows to follow
      follow_crow = c
      while follow_crow == c:
        follow_crow = random.randint(0,flock_size-1)
      
      r = random.random()
      if r >= awareness:
        for u in range(N_DEVICES):
          for k in spread_factor:
            crow_flock[c].position[u,k] = crow_flock[c].position[u,k] + random.random()*flight_length*(crow_memory[follow_crow].position[u,k]-crow_flock[c].position[u,k])
      else:
        for u in range(N_DEVICES):
          for k in spread_factor:
            crow_flock[c].position[u,k] = random.randint(*init_priority)

    #Evaluate new position of the crows
    for c in range(flock_size):
      calculate_fitness(crow_flock[c], iter, edge_nodes, G, network_data)

    #update memory of crows
    for c in range(flock_size):
      if crow_memory[c].fitness > crow_flock[c].fitness:
        crow_memory[c] = copy.deepcopy(crow_flock[c])

    #get the global best position
    for c in range(flock_size):
      if crow_memory[c].fitness < best_crow.fitness:
        best_crow = copy.deepcopy(crow_memory[c])
        converge_iter = iter
        end = time.time()

  #decode solution
  ans = decode_path(G, best_crow.position, edge_nodes)
  return [best_crow.fitness, ans, converge_iter, (end-start)]

In [ ]:
expected_ans = read_data("/content/gdrive/MyDrive/Networks/expected_ans.pkl", False)

In [ ]:
route_optimality_ratio = {}
near_route_optimality_ratio = {}
route_failure_ratio = {}
near_route_failure_ratio = {}
results = {}
percent_gap = {}
iterations_to_converge = {}
time_to_converge = {}
invalid_tests = {}
for N_DEVICES in num_of_nodes:
  correct_solution = 0
  nearly_correct = 0
  total_tests = 0
  invalid_graph = 0
  results[N_DEVICES] = []
  percent_gap[N_DEVICES] = []
  iterations_to_converge[N_DEVICES] = []
  time_to_converge[N_DEVICES] = []
  for t in range(1,test_case_num+1):

    #reading data
    network_data = {}
    G = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/graph.gml",True)
    network_data['tx_time'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/tx_time.txt", False)
    network_data['max_edge_data_rate'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/max_edge_data_rate.txt",False)
    network_data['cst'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/cst.txt",False)
    network_data['max_flow_delays'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/max_flow_delays.txt",False)
    network_data['edge_delays'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/edge_delays.txt",False)
    network_data['max_edge_delays'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/max_edge_delays.txt",False)
    network_data['data_rate'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/data_rate.txt",False)
    network_data['eh_time'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/eh_time.txt",False)
    network_data['init_res_energy'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/init_res_energy.txt",False)
    edge_nodes = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/edge_nodes.txt",False)


    #Get 2D coordinates of all the devices
    coordinates = {i:G.nodes()[i]['pos'] for i in G.nodes}

    #creating a list of bidirectional edges
    edge_list = []
    for u,v in G.edges:
      edge_list.append((u,v))
      edge_list.append((v,u))
    if expected_ans[N_DEVICES][t] == -1:
      invalid_graph = invalid_graph + 1
      continue
    optimal_cost, optimal_path, converge_iter, converge_time = Crow_Search(edge_nodes, G, network_data)
    optimal_cost = round(optimal_cost, 2)
    print(f"num_nodes = {N_DEVICES}, test_case = {t}, actual_cost = {optimal_cost}, optimal_cost = {expected_ans[N_DEVICES][t]}")

    time_to_converge[N_DEVICES].append(converge_time)
    iterations_to_converge[N_DEVICES].append(converge_iter)
    results[N_DEVICES].append([optimal_cost,expected_ans[N_DEVICES][t]])
    percent_gap[N_DEVICES].append(((optimal_cost-expected_ans[N_DEVICES][t])/expected_ans[N_DEVICES][t])*100)
    total_tests = total_tests + 1
    if ((optimal_cost-expected_ans[N_DEVICES][t])/expected_ans[N_DEVICES][t])*100 <= 4:
      nearly_correct = nearly_correct + 1
    if optimal_cost == expected_ans[N_DEVICES][t]:
      correct_solution = correct_solution + 1

  
  #calculating route optimality ratio
  if total_tests == 0:
    continue
  invalid_tests[N_DEVICES] = invalid_graph
  near_route_optimality_ratio[N_DEVICES] = nearly_correct/total_tests
  near_route_failure_ratio[N_DEVICES] = (total_tests-nearly_correct)/total_tests
  route_optimality_ratio[N_DEVICES] = correct_solution/total_tests
  route_failure_ratio[N_DEVICES] = (total_tests-correct_solution)/total_tests

In [ ]:
#saving results
def save_file(file_name, results):
  with open("/content/gdrive/MyDrive/Networks/" + file_name, 'wb') as file:
    pickle.dump(results, file)
    file.close

In [ ]:
save_file("CSA_route_optimality_ratio.pkl", route_optimality_ratio)
save_file("CSA_route_failure_ratio.pkl", route_failure_ratio)
save_file("CSA_results.pkl", results)
save_file("CSA_percent_gap.pkl", percent_gap)
save_file("CSA_iterations_to_converge.pkl", iterations_to_converge)
save_file("CSA_time_to_converge.pkl", time_to_converge)